# Time Series Prediction with LSTM Using PyTorch

This kernel is based on *datasets* from

[Time Series Forecasting with the Long Short-Term Memory Network in Python](https://machinelearningmastery.com/time-series-forecasting-long-short-term-memory-network-python/)

[Time Series Prediction with LSTM Recurrent Neural Networks in Python with Keras](https://machinelearningmastery.com/time-series-prediction-lstm-recurrent-neural-networks-python-keras/)


## Load Dataset

In [1]:
import pandas as pd
import numpy as np

# drive_path = 'drive/My Drive/Colab Notebooks/WP_Datasets_Normalized/'
# Cebrina's   Path: C:\Users\cebri\Documents\Wind Power Estimation\Data
# Guillermo's Path: C:\DTU\\02456 - Deep Learning\Project\Datasets
# Tomi's      Path: C:\Users\PC\Documents\GitHub\WindPower_Estimation

dataPath = r'C:\Users\PC\Documents\GitHub\WindPower_Estimation'

dataset_train_1 = pd.read_csv(dataPath+'\Case1\Dataset_Train_1.csv', )
dataset_test_1 = pd.read_csv(dataPath+'\Case1\Dataset_Test_1.csv')

dataset_train_1.head()

,Date_Time,Direction_10m,Speed_10m,Temperature_10m,Pressure_seaLevel,Air_Density_10m,Direction_50m,Speed_50m,Temperature_50m,Air_Density_50m,Direction_100m,Speed_100m,Temperature_100m,Air_Density_100m,Direction_150m,Speed_150m,Temperature_150m,Air_Density_150m,Park_Power_[KW]
0,2017-01-07 20:15:00,0.476323,0.110000,0.218502,0.782227,0.752212,0.473538,0.127679,0.218502,0.752212,0.484680,0.047716,0.218502,0.752212,0.515320,0.019504,0.218502,0.752212,0.690394
1,2017-01-07 20:30:00,0.479109,0.116923,0.214832,0.781496,0.755162,0.479109,0.135714,0.214832,0.755162,0.473538,0.053299,0.214832,0.755162,0.459610,0.024823,0.214832,0.755162,0.690394
2,2017-01-07 20:45:00,0.481894,0.127692,0.211009,0.780216,0.758112,0.481894,0.147321,0.211009,0.758112,0.462396,0.061421,0.211009,0.758112,0.431755,0.032624,0.211009,0.758112,0.690394
3,2017-01-07 21:00:00,0.484680,0.140769,0.207187,0.778387,0.764012,0.487465,0.161607,0.207187,0.764012,0.459610,0.072589,0.207187,0.764012,0.417827,0.042908,0.207187,0.764012,0.690394
4,2017-01-07 21:15:00,0.487465,0.155385,0.203364,0.776376,0.766962,0.490251,0.178571,0.203364,0.766962,0.456825,0.084772,0.203364,0.766962,0.412256,0.054610,0.203364,0.766962,0.690394


## Library

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn as nn
from torch.autograd import Variable
from sklearn.preprocessing import MinMaxScaler

## Data Plot

In [3]:
# Full set
training_set_X = dataset_train_1[['Speed_50m', 'Direction_50m'  ]]
training_set_Y = dataset_train_1[['Park_Power_[KW]'  ]]

test_set_X = dataset_test_1[[ 'Speed_50m', 'Direction_50m' ]]
test_set_Y = dataset_test_1[[ 'Park_Power_[KW]'  ]]

training_set_X_numpy = training_set_X.to_numpy()
training_set_Y_numpy = training_set_Y.to_numpy()
test_set_X_numpy = test_set_X.to_numpy()
test_set_Y_numpy = test_set_Y.to_numpy()




In [4]:
def custom_ts_multi_data_prep(dataset, targetset, start, window, horizon=1, end= None):
    '''
    dataset: the datas that we know and want to predict from, numpy array
    target:  the datas that we want to predict, used for error calculation
    start:   with which indice to start
    window:  how many timestamp we use from the the past, how wide is the window
    horizon: how many future target values are demanded
    '''
    X = []
    y = []
    start = start + window
    if end is None:
        end = dataset.shape[0] - horizon
    
    for i in range(start, end):
        indices = range(i-window, i)
        X.append(dataset[indices])
        indicey = range(i, i+horizon )
        y.append(targetset[indicey])
    return np.array(X), np.array(y) 

## Dataloading

In [10]:
hist_window = 5
how_many_PowerkW_values = 1
TRAIN_SPLIT = 30000
x_train, y_train = custom_ts_multi_data_prep(training_set_X_numpy,  training_set_Y_numpy,  start=0,  window=hist_window,  horizon=how_many_PowerkW_values)
x_test,  y_test  = custom_ts_multi_data_prep(test_set_X_numpy,  test_set_Y_numpy,  start=0,  window=hist_window,  horizon=how_many_PowerkW_values)


trainX = Variable(torch.Tensor(np.array(x_train)) )
trainY = Variable(torch.Tensor(np.array(y_train)) )
testX = Variable(torch.Tensor(np.array(x_test)) )
testY = Variable(torch.Tensor(np.array(y_test)) )
 
#  ----Debug----   
# print('Multiple window of past history\n')
# print( trainX[1]  )
# print('Target horizon\n')
# print(trainY.shape ) 

# print( len(trainX[1]))
# print(trainX[1].reshape(-1) ) 

In [6]:
class LSTM(nn.Module):

    def __init__(self,  num_features, num_hidden, seq_len, batch_size, num_layers=1):
        super(LSTM, self).__init__()
        
        self.num_layers = num_layers
        self.input_size = num_features
        self.seq_len = seq_len
        self.hidden_size = num_hidden
#         self.seq_length = seq_length
        
    
        # Recurrent layer
        self.lstm = nn.LSTM(input_size=num_features, hidden_size=num_hidden,
                            num_layers=num_layers, batch_first=True ) #dropout=0.1
        
        # Output layer
#         self.fc1 = nn.Linear(hidden_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, 1) #output_size=1

    
    
    def forward(self, x):
        # Initializing hidden state for first input with zeros
        h_0 = Variable(torch.zeros(
            self.num_layers, x.size(0), self.hidden_size)).to('cuda')  #.cuda()
        
#         print( x.size(0) )
#         print('================')
        # Initializing cell state for first input with zeros
        c_0 = Variable(torch.zeros(
            self.num_layers, x.size(0), self.hidden_size)).to('cuda')   #.cuda()
        
        # Propagate input through LSTM
#         print('test::::')
#         print(h_0.shape)
#         print(c_0.shape)
#         print(x.shape)
        #print(x.view(seq_len, 1))
        
#         print(f'input size: {x.shape}' )
        x_output, (hn, cn) = self.lstm( x, (h_0, c_0))   #x.view( len(x), self.seq_len, self.hidden_size)
        
#         print( x_output.shape )
#         print( x_output[0:128 , 0:1, :1].shape)
        
        lstm_out = x_output[-1].view(-1, self.hidden_size)  #h_out[-1] should be the results from the last LSTM layer
#         last_time_step = lstm_out.view(self.seq_len, len(x), self.n_hidden)[-1]
        y_pred = self.fc2(x_output )
    
        return torch.squeeze( y_pred[0:128 ,-1, :1], 1 )


In [7]:
class TimeseriesDataset(torch.utils.data.Dataset):   
    def __init__(self, X, y, seq_len=1):
        self.X = X
        self.y = y
        self.seq_len = seq_len

    def __len__(self):
        return self.X.__len__() - (self.seq_len-1)

    def __getitem__(self, index):
        return (self.X[index:index+self.seq_len], self.y[index+self.seq_len-1])
    
    
batch_siz = 100
print( training_set_X_numpy.shape)

train_dataset = TimeseriesDataset(training_set_X_numpy, training_set_Y_numpy, seq_len=8)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = batch_siz, shuffle = False)

for i, d in enumerate(train_loader):
    print(i, d[0].shape, d[1].shape)

(117764, 2)
0 torch.Size([100, 8, 2]) torch.Size([100, 1])
1 torch.Size([100, 8, 2]) torch.Size([100, 1])
2 torch.Size([100, 8, 2]) torch.Size([100, 1])
3 torch.Size([100, 8, 2]) torch.Size([100, 1])
4 torch.Size([100, 8, 2]) torch.Size([100, 1])
5 torch.Size([100, 8, 2]) torch.Size([100, 1])
6 torch.Size([100, 8, 2]) torch.Size([100, 1])
7 torch.Size([100, 8, 2]) torch.Size([100, 1])
8 torch.Size([100, 8, 2]) torch.Size([100, 1])
9 torch.Size([100, 8, 2]) torch.Size([100, 1])
10 torch.Size([100, 8, 2]) torch.Size([100, 1])
11 torch.Size([100, 8, 2]) torch.Size([100, 1])
12 torch.Size([100, 8, 2]) torch.Size([100, 1])
13 torch.Size([100, 8, 2]) torch.Size([100, 1])
14 torch.Size([100, 8, 2]) torch.Size([100, 1])
15 torch.Size([100, 8, 2]) torch.Size([100, 1])
16 torch.Size([100, 8, 2]) torch.Size([100, 1])
17 torch.Size([100, 8, 2]) torch.Size([100, 1])
18 torch.Size([100, 8, 2]) torch.Size([100, 1])
19 torch.Size([100, 8, 2]) torch.Size([100, 1])
20 torch.Size([100, 8, 2]) torch.Size(

250 torch.Size([100, 8, 2]) torch.Size([100, 1])
251 torch.Size([100, 8, 2]) torch.Size([100, 1])
252 torch.Size([100, 8, 2]) torch.Size([100, 1])
253 torch.Size([100, 8, 2]) torch.Size([100, 1])
254 torch.Size([100, 8, 2]) torch.Size([100, 1])
255 torch.Size([100, 8, 2]) torch.Size([100, 1])
256 torch.Size([100, 8, 2]) torch.Size([100, 1])
257 torch.Size([100, 8, 2]) torch.Size([100, 1])
258 torch.Size([100, 8, 2]) torch.Size([100, 1])
259 torch.Size([100, 8, 2]) torch.Size([100, 1])
260 torch.Size([100, 8, 2]) torch.Size([100, 1])
261 torch.Size([100, 8, 2]) torch.Size([100, 1])
262 torch.Size([100, 8, 2]) torch.Size([100, 1])
263 torch.Size([100, 8, 2]) torch.Size([100, 1])
264 torch.Size([100, 8, 2]) torch.Size([100, 1])
265 torch.Size([100, 8, 2]) torch.Size([100, 1])
266 torch.Size([100, 8, 2]) torch.Size([100, 1])
267 torch.Size([100, 8, 2]) torch.Size([100, 1])
268 torch.Size([100, 8, 2]) torch.Size([100, 1])
269 torch.Size([100, 8, 2]) torch.Size([100, 1])
270 torch.Size([100,

444 torch.Size([100, 8, 2]) torch.Size([100, 1])
445 torch.Size([100, 8, 2]) torch.Size([100, 1])
446 torch.Size([100, 8, 2]) torch.Size([100, 1])
447 torch.Size([100, 8, 2]) torch.Size([100, 1])
448 torch.Size([100, 8, 2]) torch.Size([100, 1])
449 torch.Size([100, 8, 2]) torch.Size([100, 1])
450 torch.Size([100, 8, 2]) torch.Size([100, 1])
451 torch.Size([100, 8, 2]) torch.Size([100, 1])
452 torch.Size([100, 8, 2]) torch.Size([100, 1])
453 torch.Size([100, 8, 2]) torch.Size([100, 1])
454 torch.Size([100, 8, 2]) torch.Size([100, 1])
455 torch.Size([100, 8, 2]) torch.Size([100, 1])
456 torch.Size([100, 8, 2]) torch.Size([100, 1])
457 torch.Size([100, 8, 2]) torch.Size([100, 1])
458 torch.Size([100, 8, 2]) torch.Size([100, 1])
459 torch.Size([100, 8, 2]) torch.Size([100, 1])
460 torch.Size([100, 8, 2]) torch.Size([100, 1])
461 torch.Size([100, 8, 2]) torch.Size([100, 1])
462 torch.Size([100, 8, 2]) torch.Size([100, 1])
463 torch.Size([100, 8, 2]) torch.Size([100, 1])
464 torch.Size([100,

731 torch.Size([100, 8, 2]) torch.Size([100, 1])
732 torch.Size([100, 8, 2]) torch.Size([100, 1])
733 torch.Size([100, 8, 2]) torch.Size([100, 1])
734 torch.Size([100, 8, 2]) torch.Size([100, 1])
735 torch.Size([100, 8, 2]) torch.Size([100, 1])
736 torch.Size([100, 8, 2]) torch.Size([100, 1])
737 torch.Size([100, 8, 2]) torch.Size([100, 1])
738 torch.Size([100, 8, 2]) torch.Size([100, 1])
739 torch.Size([100, 8, 2]) torch.Size([100, 1])
740 torch.Size([100, 8, 2]) torch.Size([100, 1])
741 torch.Size([100, 8, 2]) torch.Size([100, 1])
742 torch.Size([100, 8, 2]) torch.Size([100, 1])
743 torch.Size([100, 8, 2]) torch.Size([100, 1])
744 torch.Size([100, 8, 2]) torch.Size([100, 1])
745 torch.Size([100, 8, 2]) torch.Size([100, 1])
746 torch.Size([100, 8, 2]) torch.Size([100, 1])
747 torch.Size([100, 8, 2]) torch.Size([100, 1])
748 torch.Size([100, 8, 2]) torch.Size([100, 1])
749 torch.Size([100, 8, 2]) torch.Size([100, 1])
750 torch.Size([100, 8, 2]) torch.Size([100, 1])
751 torch.Size([100,

934 torch.Size([100, 8, 2]) torch.Size([100, 1])
935 torch.Size([100, 8, 2]) torch.Size([100, 1])
936 torch.Size([100, 8, 2]) torch.Size([100, 1])
937 torch.Size([100, 8, 2]) torch.Size([100, 1])
938 torch.Size([100, 8, 2]) torch.Size([100, 1])
939 torch.Size([100, 8, 2]) torch.Size([100, 1])
940 torch.Size([100, 8, 2]) torch.Size([100, 1])
941 torch.Size([100, 8, 2]) torch.Size([100, 1])
942 torch.Size([100, 8, 2]) torch.Size([100, 1])
943 torch.Size([100, 8, 2]) torch.Size([100, 1])
944 torch.Size([100, 8, 2]) torch.Size([100, 1])
945 torch.Size([100, 8, 2]) torch.Size([100, 1])
946 torch.Size([100, 8, 2]) torch.Size([100, 1])
947 torch.Size([100, 8, 2]) torch.Size([100, 1])
948 torch.Size([100, 8, 2]) torch.Size([100, 1])
949 torch.Size([100, 8, 2]) torch.Size([100, 1])
950 torch.Size([100, 8, 2]) torch.Size([100, 1])
951 torch.Size([100, 8, 2]) torch.Size([100, 1])
952 torch.Size([100, 8, 2]) torch.Size([100, 1])
953 torch.Size([100, 8, 2]) torch.Size([100, 1])
954 torch.Size([100,

In [12]:
num_epochs = 100
learning_rate = 0.002
hist_window 
n_features = 2
hidden_size = 20
num_layers = 1

batch_size = 672
num_samples_train = len( training_set_X_numpy )
num_batches_train = (num_samples_train // batch_size ) 


#-----------------------------------------------------------------------------------
print(f'shape training: {( training_set_X_numpy.shape)}')
remainder_train = len( training_set_X_numpy)%batch_size
print(f'training remainder: {remainder_train}')

for rem in range( remainder):
    training_set_X_numpy = np.delete( training_set_X_numpy, 0, axis=0)
    
remainder2_train = len( training_set_X_numpy)%batch_size
print(f'training remainder: {remainder2_train}')
print(f'shape training2: { ( training_set_X_numpy.shape)}')
#------------------------------------------------------------------------------------

print()
print(f'shape test: {( test_set_X_numpy.shape)}')
remainder_test = len( test_set_X_numpy)%batch_size
print(f'test remainder: {remainder_test}')
for rem in range( remainder_test):
    test_set_X_numpy = np.delete( test_set_X_numpy, 0, axis=0)
remainder2_test = len( test_set_X_numpy)%batch_size
print(f'test remainder: {remainder2_test}')
print(f'shape training2: { ( training_set_X_numpy.shape)}')
#------------------------------------------------------------------------------------
print()



lstm = LSTM(num_features=n_features, num_hidden=hidden_size, seq_len=hist_window, num_layers=num_layers, batch_size=batch_size )
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

use_cuda = torch.cuda.is_available()
if use_cuda:
    lstm.to(device)
    print('run on cuda')
    print( next(lstm.parameters()).is_cuda )
print(lstm)

criterion = torch.nn.MSELoss()    # mean-squared error for regression
optimizer = torch.optim.Adam(lstm.parameters(), lr=learning_rate)
#optimizer = torch.optim.SGD(lstm.parameters(), lr=learning_rate)

# Train the model
for epoch in range(num_epochs):
    
    batch_siz = 128
    train_dataset = TimeseriesDataset(training_set_X_numpy, training_set_Y_numpy, seq_len=8) 
    train_loader = iter( torch.utils.data.DataLoader(train_dataset, batch_size = batch_siz, shuffle = False))
    
    lstm.train()
    print( f'new epoch, number{epoch}')
    
    # For each sentence in training set
    for index in range( num_batches_train ):
        optimizer.zero_grad()
        x, y = train_loader.next()
        x= x.to('cuda')
        y= y.to('cuda') 
        
#         print(index)
#         print( x.shape )

        outputs = lstm(  x.float() )


        # obtain the loss function
#         print('++++++++')
#         print(outputs.shape)
# #         print(outputs )
# #         print(outputs)
#         print(y.shape)
        loss = criterion(outputs, y.float())

        loss.backward()

        optimizer.step()

    print("Epoch: %d, loss: %1.5f" % (epoch, loss.item()))

shape training: (117436, 2)
training remainder: 508
training remainder: 344
shape training2: (117272, 2)

shape test: (28896, 2)
test remainder: 0
test remainder: 0
shape training2: (117272, 2)
run on cuda
True
LSTM(
  (lstm): LSTM(2, 20, batch_first=True)
  (fc2): Linear(in_features=20, out_features=1, bias=True)
)
new epoch, number0
Epoch: 0, loss: 0.17960
new epoch, number1
Epoch: 1, loss: 0.17112
new epoch, number2
Epoch: 2, loss: 0.16613
new epoch, number3
Epoch: 3, loss: 0.16038
new epoch, number4
Epoch: 4, loss: 0.15676
new epoch, number5
Epoch: 5, loss: 0.15377
new epoch, number6
Epoch: 6, loss: 0.15145
new epoch, number7
Epoch: 7, loss: 0.14840
new epoch, number8
Epoch: 8, loss: 0.14586
new epoch, number9
Epoch: 9, loss: 0.14400
new epoch, number10
Epoch: 10, loss: 0.14252
new epoch, number11
Epoch: 11, loss: 0.14108
new epoch, number12
Epoch: 12, loss: 0.13989
new epoch, number13
Epoch: 13, loss: 0.13909
new epoch, number14
Epoch: 14, loss: 0.13846
new epoch, number15
Epoch: 

## Dataloading

## Model

## Training

## Testing for Park Power values

In [9]:
pred_testX = []

# Predicting first values of the test data with last values of train data
# Each iteration predicts exactly 1 value and adds it to the pred_testX array
# When items for train data cannot be fed into the LSTM, items from pred_testX are used, one by one

window_test_set = np.vstack((training_set[len(training_set)-seq_length:len(training_set)],[[0],[0]]))
x, y = sliding_windows(window_test_set, seq_length)
dataX = Variable(torch.Tensor(np.array(x)))

lstm.eval()
test_predict = lstm(dataX)
data_predict = test_predict.data.numpy()
pred_testX.append(data_predict[0])

for i in range(1,seq_length):
  window_test_set = np.vstack((training_set[len(training_set)-seq_length+i:len(training_set)],pred_testX,[[0],[0]]))
  x, y = sliding_windows(window_test_set, seq_length)
  dataX = Variable(torch.Tensor(np.array(x)))

  lstm.eval()
  test_predict = lstm(dataX)
  data_predict = test_predict.data.numpy()
  pred_testX.append(data_predict[0])

# Predicting rest of the 96 values exclusively with predicted data from pred_testX
for i in range(96-seq_length):
  window_test_set = np.vstack((pred_testX[len(pred_testX)-seq_length:len(pred_testX)],[[0],[0]]))
  x, y = sliding_windows(window_test_set, seq_length)
  dataX = Variable(torch.Tensor(np.array(x)))

  lstm.eval()
  test_predict = lstm(dataX)
  data_predict = test_predict.data.numpy()
  pred_testX.append(data_predict[0])

NameError: name 'training_set' is not defined

In [ ]:
# Plotting last train values with test values vs predicted values

lstm.eval()
train_predict = lstm(trainX)

train_predict = train_predict.data.numpy()

dataY_plot = np.vstack((training_set[len(training_set)-224:],test_set[0:97]))
data_predict = np.vstack((train_predict[len(train_predict)-224:],pred_testX))

plt.axvline(x=len(training_set[len(training_set)-224:]), c='r', linestyle='--')

plt.plot(dataY_plot)
plt.plot(data_predict)
plt.suptitle('Time-Series Prediction')
plt.show()